<a href="https://colab.research.google.com/github/chanhee922/DeepLearning_Practice/blob/master/%EB%8B%A4%EC%B8%B5%EC%8B%A0%EA%B2%BD%EB%A7%9D_%EC%BD%94%EB%94%A9%EC%9D%84_%ED%86%B5%ED%95%9C_%EC%9D%B4%ED%95%B4!!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2개층을 가진 다층신경망
## (입력층 1개, 은닉층 1개, 출력층 1개)

### - 입력층 shape: 입력 특성의 개수
### - 은닉층 shape: hyper-parameter(뉴런의 개수는 직접 결정)
### - 출력층 shape: 출력층의 노드 개수 = 1

In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# SingleLayer class (단층신경망구현)에 배치입력을 통한, 
# 배치경사하강법(Batch Gradient Descent) 적용
class SingleLayer:
    
    def __init__(self, learning_rate=.1, l1=0, l2=0): # 학습률(에타) 하이퍼파라미터 추가
        self.w = None
        self.b = None        
        
        # 손실함수의 평균오차값을 매 훈련시마다(epoch) 저장
        self.losses = []       # 훈련셋의 손실값을 저장하는 리스트
        self.val_losses = []   # 검증셋의 손실값을 저장하는 리스트
        
        self.w_history = []      # NEW: 매훈련시마다, 학습으로 변하는 가중치 저장
        
        self.lr   = learning_rate  # NEW: 학습률 저장
        
        # L1/L2 규제강도를 조절하는 alpha 값 저장
        self.l1 = l1    # L1 norm
        self.l2 = l2    # L2 norm
        
        pass # constructor
    
    
    # 정방향계산(=뉴런의 선형함수 계산)
    def forpass(self, x):  
        # X : 2-D, Matrix
        # self.w : 2-D, Matrix
        # z = w1x1 + w2x2 + ..... wnxn + b
        
#         z = np.sum(x * self.w) + self.b
        z = np.dot(x, self.w) + self.b
        return z
        
        pass # forpass
    
    
    # 역방향계산(오차역전파와 손실함수의 미분을 통한 가중치와 절편의 변화율 계산)
    def backprop(self, x, err):
        m = len(x)   # 전체샘플크기(m)를 구합니다.
        
#         w_grad = x * err    
#         b_grad = 1 * err    
        
        # 가중치(w)에 대한 변화율(=gradient) "평균" 계산
        w_grad = np.dot(x.T, err) / m  # m: 전체샘플크기       
        
        # 절편(b)에 대한 변화율(=gradient) "평균" 계산
        b_grad = np.sum(err) / m   #  m: 전체샘플크기    
    
        return w_grad, b_grad
        pass # backprop
    
    
    # 활성화함수(시그모이드) 통과하여 반환
    def activation(self, z):
        # **** 안전한 지수(또는 로그) 계산을 위해, 지정됨 범위 값을 가지도록 함
#         z = np.clip(z, 1e-10, 1 - 1e-10)   # 1st. method
        z = np.clip(z, -100, None)         # 2nd. method
    
        a = 1 / (1 + np.exp(-z))
        return a
        pass # activation
    
    
    # 신경망 훈련을 통한 최적의 가중치(w)와 절편(b)을
    # 찾아가도록 훈련
    def fit(self, x, y, epochs=100, x_val=None, y_val=None):
        m = len(y)
        
        # (0). 정답을 열벡터 형태의 모양으로 reshaping
        y = y.reshape(-1, 1)         # 훈련셋의 정답
        y_val = y_val.reshape(-1, 1) # 검증셋의 정답                     
        
        # (1). 가중치와 절편의 초기값 설정
        # 2-D, Matrix with all elements is 1
        self.w = np.ones( (x.shape[1], 1) )  
        self.b = 0
        
        # 초기 가중치 값 리스트에 추가
        self.w_history.append(self.w.copy()) 
        
        # (2)  epoch 돌립니다.
        for i in range(epochs):
            
            # (3) 전체샘플을 활용한 학습(가중치와 절편을 업데이트) 수행
            #
            # 신경망을 훈련시킬 훈련 데이터 셋은, 있는 그대로 밀어넣는게 아니라,
            # 매 에포크(epoch) 마다, shuffling해서, 밀어 넣어야 함!!!
            # 전체 sample 을 무작위로 섞는 shuffling 함수 ==> np.random.permutation()함수
            
            indexes = np.arange( len(x) )  # 전체 샘플의 index 번호를 1-D, Vector로 생성
            indexes = np.random.permutation( indexes  ) # shuffling 후 다시 저장
            
            # 매 epoch 마다 평균 손실함수의 오차값 산출하여, 리스트에 저장
            loss = 0
            
            # 안쪽의 매 샘플마다 반복시키는 loop는 삭제!!!
            # 왜? 한번입력이 전체 샘플이기 때문에....
            
#             xi = x[i]   # 매번 샘플의 입력데이타
#             yi = y[i]   # 매번 샘플의 정답데이타

            # (4) 정방향계산 수행
            z = self.forpass(x)

            # (5) 활성화함수 통과
            a = self.activation(z)

            # (6) 오차계산
            err = -(y - a)

            # (7) 오차역전파를 통한, 가중치와 절편의 변화율 계산
            w_grad, b_grad = self.backprop(x, err)

            # (***) L1/L2 " 평균"규제를 적용
            w_grad += ( self.l1 * np.sign(self.w) + self.l2 * self.w ) / m

            # (8) 계산된 가중치와 절편의 변화율을 이용한, 가중치/절편의 업데이트
            # NEW: 가중치의 업데이트 양을 조절하는 하이퍼파라미터인 "학습률(에타)" 적용
            self.w -= self.lr * w_grad  
            self.b -= b_grad

            # NEW: 매번 업데이트 되는 가중치를 파이썬 리스트에추가
            self.w_history.append(self.w.copy()) # Deep copy

            # (9) 로지스틱 손실함수의 계산 수행
            # 안전한 로그계산을 위한 클리핑 수행
            a = np.clip(a, 1e-10, 1 - 1e-10) # 1st.method
#           a = np.clip(a, -100, None)       # 2st.method

            L = np.sum( -( y * np.log(a) + (1 - y) * np.log(1-a) ) )
            loss += L
                
#                 pass # traninig loop (1 epoch)
            
            # 로지스틱손실함수의 평균값 계산
            self.losses.append( ( loss + self.reg_loss() )  / m )

            # 검증 세트에 대해서도 손실을 계산
            self.update_val_loss(x_val, y_val)
            
            pass # epoch loop
        
        pass # fit
    
    # L1/L2 규제의 손실값 계산
    def reg_loss(self):
        # L1 norm 의 정의식 대로 계산
        # L1 규제의 미분결과 = a x sign(w)
        L1_loss = self.l1 * np.sum(np.abs(self.w))
        (self.l1 * np.sign(self.w1) + self.l2 * self.w1) / m
        # L2 norm 의 정의식 대로 계산
        # L2 규제의 미분결과 = a x w
        L2_loss = self.l2 / 2 * np.sum(self.w**2)
        
        return  L1_loss + L2_loss                
        pass # reg_loss
    
    
    # 검증 데이터 셋에 대해서도, 동일하게
    # 손실함수를 계산하고, L1/L2 규제도 적용하여, 리스트에 저장
    def update_val_loss(self, x_val, y_val):        
        z = self.forpass(x_val) # 정방향 계산 수행
        a = self.activation(z)     # 활성화 함수 통과
        a = np.clip(a, 1e-10, 1-1e-10)
            
        # 검증셋에 대한 손실함수의 값을 계산
        # (1) Logistic 손실함수의 값을 계산
        val_loss = np.sum(-(y_val * np.log(a) + (1-y_val) * np.log(1-a)))
                    
        # 검증셋의 평균손실값을 계산하여, 파이썬 리스트(val_losses)에 추가
        # 더불어서, L1/L2 규제의 손실값을 계산하여 추가
        self.val_losses.append( 
            ( val_loss + self.reg_loss() ) / len(y_val)
        )
        
        pass # update_val_loss
    
    
    # 분류예측함수
    def predict(self, x):
        # 정방향계싼 수행
        z = self.forpass(x)
        
        # 간결성과 가독성을 위해서는, 죽~아래코드를 그대로 활용
        # if z > 0, sigmoid(z) > 0.5
        # if z < 0, sigmoid(z) < 0.5
#         return np.array(z) > 0 # 1-D, Vector
        
        # 활성화함수 통과
        # np.array(z) : python list -> ndarray (1-D, Vector) 변환
        a = self.activation(np.array(z))  
        
        # 임계함수로 step function 통과시켜, 최종 분류예측값 산출
        return a > .5
        pass    # predict
    
    # 모델의 성능(정확도)평가 함수
    def score(self, x, y):
        return np.mean(self.predict(x) == y.reshape(-1, 1))
        pass # score

    pass # end class

In [0]:
class DualLayer(SingleLayer):

    # 각종 하이퍼 파아미터와 가중치와 절편 행렬의 초기화
    def __init__(self, units = 10, learning_rate = .1, l1 = 0, l2 = 0):
        self.units = units

        # 은닉층의 가중치(w)와 절편(b)를 초기화
        self.w1 = None
        self.b1 = None

        # 출력층의 가중치(w)와 절편(b)를 초기화
        self.w2 = None
        self.b2 = None

        self.losses = []
        self.val_losses = []

        self.lr = learning_rate     # eta

        self.l1 = l1
        self.l2 = l2

        self.a1 = None
        pass    # constructor
    
    # 정방향 계산
    def forpass(self, x):
        
        # 은닉층의 선형함수 출력값 계산
        z1 = np.dot(x, self.w1) + self.b1
        self.a1 = self.activation(z1)       # 활성화

        # 출력층의 선형함수 출력값 계산
        z2 = np.dot(self.a1, self.w2) + self.b1

        return z2
        pass    # forpass

    # 역전파된 오차를 가지고 입력층 이전 계층('은닉층')까지의 모든 계층의
    # 가중치와 절편의 '평균' 변화율을 계산 및 반환
    def backprop(self, x, err):     # x, err: 각각 다 행렬
        m = len(x)      # 전체 샘플의 크기 확보

        # '출력층'의 가중치(w)와 절편(b)의 평균 변화율 계산
        w2_grad = np.dot(self.a1.T, err) / m
        b2_grad = np.sum(err) / m

        # '은닉층'의 가중치(w1)와 절편(b1)의 평균 변화율 계산
        temp = np.dot(self.w1.T, err) * self.a1 * (1 - self.a1)

        w1_grad = np.dot(x.T, temp) / m
        b1_grad = np.sum(temp, axis=0) / m
        pass        # backprop

    # 신경망 각 층의 가중치와 절편을 초기화
    def init_weights(self, n_features):
        # 은닉층의 가중치와 절편 초기화
        self.w1 = np.ones( (n_features, self.units) )
        self.b1 = np.zeros(self.units)

        # 출력층의 가중치와 절편 초기화
        self.w2 = np.ones((self.units, 1))
        self.b2 = 0

        pass        # init_weights

    # 신경망의 모델을 훈련시키는 핵심 함수
    def training(self, x, y, m):
        # 1. 정방향 계산 수행
        z = self.forpass(x)

        # 2. 활성화 출력값 계산 수행
        a = self.activation(z)

        # 3. 오차 계산
        err = -(y - a)

        # 4. 오차 역전파를 통한 각 층의 가중치와 절편 변화율 계산
        w1_grad, b1_grad, w2_grad, b2_grad = self.backprop(x, err)

        # 5. L1/L2 규제의 미분값을 가중치의 변화율에 더하고 평균 계산
        w1_grad += (self.l1 * np.sign(self.w1) + self.l2 * self.w1) / m
        w2_grad += (self.l1 * np.sign(self.w2) + self.l2 * self.w2) / m
        
        # 6. '은닉층', '출력층'의 가중치와 절편을 업데이트(1 epoch마다)
        self.w1 -= self.lr * w1_grad
        self.b1 -= self.lr * b1_grad

        self.w2 -= self.lr * w2_grad
        self.b2 -= self.lr * b2_grad

        return a
        pass        # training

    # '은닉층'과 '출력층'의 L1/L2 규제 손실 계산
    def reg_loss(self):

        return self.l1 * ( np.sum( np.abs(self.w1) ) ) + np.sum( np.abs(self.w2) ) + self.l2 / 2 * ( np.sum(self.w1**2) + np.sum(self.w2**2) )
        pass # reg_loss


    def fit(self, x, y, epochs=100, x_val=None, y_val=None):
        # 분석가가 제공한 정답(훈련셋/검증셋의 정답)을
        # 열 벡터 모양의 행렬로 변환
        y = y.reshape(-1, 1)

        # 전체샘플크기 확보
        m = len(x)

        #  가중치와 절편의 최초 초기화 수행
        self.init_weights(x.shape[1])

        # epoch loop
        for i in range(epochs):
            # 정방향 계산, 오차역전파, 역방향계산,
            # L1/L2 규제 적용 등을 통해, 각 계층의
            # 가중치와 절편의 변화율을 구하고 업데이트 수행
            a = self.training(x, y, m) # *******
            a = np.clip(a, 1e-10, 1-1e-10)
            # L (손실함수)의 손실과 L1/L2 규제 손실을 계산하여
            # 파이썬 리스트에 추가
            loss = np.sum( -( y * np.log(a) + (1-y) * np.log(1-a) ) )
            L_1_2 = self.reg_loss()

            self.losses.append( ( loss + L_1_2 ) / m )

            # 검증셋에 대한 손실값 계산 후 파이썬 리스트에 추가
            self.update_val_loss(x_val, y_val)
            pass        # epoch loop

        pass        # fit

In [0]:
dualNN = DualLayer(l2 = .01)
dualNN

In [0]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
x = cancer.data
y = cancer.target

# x.shape, y.shape

# (1-1) 1단계 분할 : 8:2 로, 훈련:테스트로 분할
from sklearn.model_selection import train_test_split

x_train_all, x_test_all, y_train_all, y_test_all = \
    train_test_split(x, y, test_size=.2, stratify=y)

# (1-2) 2단계 분할 : 1단계의 분할된 훈련 데이터 셋을, 다시
#                    8:2 로, 훈련:검증로 분할
x_train, x_val, y_train, y_val = \
    train_test_split(
        x_train_all, 
        y_train_all, 
        test_size=.2,
        stratify=y_train_all
    )

In [0]:
import numpy as np

# 입력데이터에 대한 표준화 전처리 수행 - 2st. method (직접구현)
x_train.shape

# 표준화 전처리 수행
# (1) 평균 산출 > (2) 표준편차 산출 > (3) 공식에 따라 표준화 수행

# 30개의 각 특성별 평균을 구하라!!
x_train_mean = np.mean(x_train, axis=0) # 평균

# 30개의 각 특성별 표준편차를 구하라!!!
x_train_std = np.std(x_train, axis=0)   # 표준편차

x_train_scaled = ( x_train - x_train_mean ) / x_train_std
x_train_scaled.shape

(364, 30)

In [0]:
# 여러 셋(훈련/검증/테스트)으로 나누어진, 각각의 셋을 표준화시킬때에는
# 훈련셋의 통계량(평균, 표준편차)를 기준으로 나머지 검증/테스트 셋도
# 표준화 시켜야 함

# x_train_mean ---> 훈련셋의 평균
# x_train_std  ---> 훈련셋의 표준편차

x_val_scaled = (x_val - x_train_mean) / x_train_std
# x_test_scaled = (x_test - x_train_mean ) / x_train_std

In [0]:
x_train_scaled.shape, x_val_scaled.shape, y_train.shape, y_val.shape

((364, 30), (91, 30), (364,), (91,))

In [0]:
dualNN.fit(x_train_scaled, y_train, x_val = x_val_scaled, y_val=y_val, epochs = 20000)
dualNN.score(x_val_scaled, y_val)

ValueError: ignored